In [1]:
import io

import random
import string

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('raw_data/201909_long.csv')
location = pd.read_csv('LOCATION.csv')
location.head()

,LocationID,Name,Building,Room,FullCapacity,ReducedCapacity,Latitude,Longitude
0,Loc_81644,GREENE 204,GREENE BUILDING,204,20,13,42.7298,73.6789
1,Loc_08237,GREENE 302,GREENE BUILDING,302,2,6,42.7298,73.6789
2,Loc_41662,WEST 112,WEST HALL,112,24,14,42.7298,73.6789
3,Loc_54058,PEOPLES 4204,PEOPLES AVE COMPLEX J,4204,16,6,42.7298,73.6789
4,Loc_43449,PEOPLES 4208,PEOPLES AVE COMPLEX J,4208,16,6,42.7298,73.6789


In [3]:
def id_generator(size = 10, chars = string.digits):
  return ''.join(random.choice(chars) for _ in range(size))

In [4]:
events = df.copy()
# Drop classes with TBA times
events = events[events["start"] != '** TBA **']

# Consider non admin classes only 
events = events[events["subject"] != 'ADMN']

In [5]:
# Drop duplicates
events = events.drop_duplicates().reset_index(drop = True)
#events = events.drop_duplicates(subset = 'name').reset_index(drop = True)

# Set ID
events["EventID"] = ["Event_" + id_generator() for i in range(events.shape[0])]

# Drop 'weekly_position', 'subject', 'courseno', 'section', 'location', 'seats_taken', and 'seats'
events = events.drop(['weekly_position', 'subject', 'courseno', 'location', 'seats_taken', 'seats'], axis = 1)

# Sort by time
events = events.sort_values(by = ['day','start']).reset_index(drop = True)

# Show the resultant dataset
events

,crn,name,section,start,end,day,EventID
0,81215,Air And Space Studies 300 A,01,0700,0950,1,Event_1943344516
1,81216,Air And Space Studies 400 A,01,0700,0950,1,Event_5790671382
2,82332,Molecular Biology Ii,01,0800,0950,1,Event_3641784369
3,82905,Adv Molecular Biology,01,0800,0950,1,Event_7754785436
4,84400,Materials Science,01,0800,0950,1,Event_2068553977
...,...,...,...,...,...,...,...
3273,85396,Rcos Small Group Meetings,08,1600,1750,5,Event_5656776249
3274,83597,Rcos Small Group Meetings,10,1600,1750,5,Event_4133381614
3275,85525,The American Dream,02,1600,1750,5,Event_9675463691
3276,84910,Writing In Context,03,1600,1750,5,Event_4788731260


In [6]:
for day in range(1,6):
    for index, row in events[events['day'] == day].iterrows():
        events.loc[index, "LocationID"] = str(location.loc[index%(location.shape[0]), 'LocationID']).split('.')[0]
        events.loc[index, "Seats"] = location.loc[index%(location.shape[0]), 'ReducedCapacity']
        events.loc[index, "SeatsTaken"] = 0
        
events['Seats'] = pd.to_numeric(events['Seats'], downcast = 'signed')
events['SeatsTaken'] = pd.to_numeric(events['SeatsTaken'], downcast = 'signed')
events['start'] = pd.to_numeric(events['start'], downcast = 'signed')
events['end'] = pd.to_numeric(events['end'], downcast = 'signed')
events = events[["crn","EventID", "section", "name", "day", "start", "end", "SeatsTaken", "Seats", "LocationID"]]
events = pd.merge(events, location, on = 'LocationID', how = 'outer')[["crn","EventID", "section", "name", "day", "start", "end", "SeatsTaken", "Seats", "LocationID","Building"]]
events.columns = ["CRN","EventID", "Section", "Name", "DayOfWeek", "StartTime", "EndTime", "SeatsTaken", "Seats", "LocationID", "Building"]
events["Type"] = "Class"
events = events.drop_duplicates(subset = ['CRN', 'Section', 'DayOfWeek', 'StartTime', 'EndTime'], keep = 'first')
events.head()

,CRN,EventID,Section,Name,DayOfWeek,StartTime,EndTime,SeatsTaken,Seats,LocationID,Building,Type
0,81215,Event_1943344516,01,Air And Space Studies 300 A,1,700,950,0,13,Loc_81644,GREENE BUILDING,Class
1,85313,Event_7152077997,01,Programming For Bme,1,1000,1150,0,13,Loc_81644,GREENE BUILDING,Class
2,81136,Event_4786369640,01,Introduction To Optimization,1,1000,1150,0,13,Loc_81644,GREENE BUILDING,Class
3,85230,Event_3240167821,01,Sedimentology /Stratigraphy,1,1200,1350,0,13,Loc_81644,GREENE BUILDING,Class
4,84417,Event_6000328461,03,Architectural Design Studio 7,1,1400,1750,0,13,Loc_81644,GREENE BUILDING,Class


In [7]:
# Generate ID for each person
persons = pd.DataFrame({"PersonID": ["Person_" + id_generator(size = 9) for i in range(1700)]})

# Select their address from a list of addresses
addresses = ['Barton Hall', 'Bray Hall', 'Cary Hall', 
             'Blitman Residence Commons', 'City Station South', 
             'Polytechnic Apartments', 'Beman and Brinsmade']
persons["Address"] = [random.choice(addresses) for _ in range(1700)]
persons["Test"] = 'NA'
# Show the top rows
persons.head()

,PersonID,Address,Test
0,Person_729960822,Bray Hall,NA
1,Person_890908145,Blitman Residence Commons,NA
2,Person_332261652,Cary Hall,NA
3,Person_076194812,Blitman Residence Commons,NA
4,Person_135788187,Blitman Residence Commons,NA


In [8]:
persons.to_csv("PERSON.csv", index = False)

In [9]:
build_transitions = {
    "RUSSELL SAGE LABORATORY": {"Entry": [["SAGE FRONT ENT", 0.7], ["SAGE SIDE ENT",0.3]],
                                "Exit": [["SAGE BACK EXIT", 0.7], ["SAGE SIDE EXIT", 0.3]]},
    "AMOS EATON HALL": {"Entry": ["AE FRONT ENT"], "Exit": ["AE BACK EXIT"]},
    "CARNEGIE BUILDING": {"Entry": ["CARN FRONT ENT"], "Exit": ["CARN BACK EXIT"]},
    "DARRIN COMMUNICATIONS CENTER": {"Entry": ["DCC FRONT ENT"], "Exit": ["DCC BACK EXIT"]},
    "ACADEMY HALL": {"Entry": ["ACAD FRONT ENT"], "Exit": ["ACAD BACK EXIT"]},
    "PITTSBURGH BUILDING": {"Entry": ["PITT FRONT ENT"], "Exit": ["PITT BACK EXIT"]},
    "TROY BUILDING": {"Entry": ["TROY FRONT ENT"], "Exit": ["TROY BACK EXIT"]},
    "WALKER LABORATORY": {"Entry": ["WALKER FRONT ENT"], "Exit": ["WALKER BACK EXIT"]},
    "GREENE BUILDING": {"Entry": ["GREENE FRONT ENT"], "Exit": ["GREENE BACK EXIT"]},
    "RICKETTS BUILDING": {"Entry": ["RICKETTS FRONT ENT"], "Exit": ["RICKETTS BACK EXIT"]},
    "JONSSON ENGINEERING CENTER": {"Entry": ["JEC FRONT ENT"], "Exit": ["JEC BACK EXIT"]},
    "JONSSON-ROWLAND SCIENCE CENTER": {"Entry": ["JRSC FRONT ENT"], "Exit": ["JRSC BACK EXIT"]},
    "LOW CENTER FOR INDUSTRIAL INNOVATION": {"Entry": ["LOW FRONT ENT"], "Exit": ["LOW BACK EXIT"]},
    "VOORHEES COMPUTING CENTER": {"Entry": ["VCC FRONT ENT"], "Exit": ["VCC BACK EXIT"]},
    "MATERIALS RESEARCH CENTER": {"Entry": ["MRC FRONT ENT"], "Exit": ["MRC BACK EXIT"]},
    "AS&RC": {"Entry": ["AS&RC FRONT ENT"], "Exit": ["AS&RC BACK EXIT"]},
    "WEST HALL": {"Entry": ["WEST FRONT ENT"], "Exit": ["WEST BACK EXIT"]},
    "NASON HALL": {"Entry": ["NASON FRONT ENT"], "Exit": ["NASON BACK EXIT"]},
    "LALLY HALL": {"Entry": ["LALLY FRONT ENT"], "Exit": ["LALLY BACK EXIT"]},
    "CARY HALL": {"Entry": ["CARY FRONT ENT"], "Exit": ["CARY BACK EXIT"]}
    #"HOUSTON FIELD HOUSE": {"Entry": ["HFH FRONT ENT"], "Exit": ["HFH BACK EXIT"]}
    
}
transition = events.copy()[['Building', 'DayOfWeek', 'StartTime', 'EndTime']]
transition.drop_duplicates(keep = 'first', inplace = True)
entry = transition.copy()
entry["EndTime"] = entry["StartTime"]
exit = transition.copy()
exit["StartTime"] = exit["EndTime"]
#entry.head()
transition.head()
               

,Building,DayOfWeek,StartTime,EndTime
0,GREENE BUILDING,1,700,950
1,GREENE BUILDING,1,1000,1150
3,GREENE BUILDING,1,1200,1350
4,GREENE BUILDING,1,1400,1750
5,GREENE BUILDING,1,1400,1550


In [10]:
entry_mod = pd.DataFrame(columns = ['Building', 'DayOfWeek', 'StartTime', 'EndTime'])
exit_mod = pd.DataFrame(columns = ['Building', 'DayOfWeek', 'StartTime', 'EndTime'])

for index, row in entry.iterrows():
    name = 'NA'
    if(row["EndTime"] % 100 == 0):
        interval = 50
    else:
        interval = 10
    if(row["Building"] in build_transitions):
        if(len(build_transitions[row['Building']]["Entry"]) == 1):
            entry_mod = entry_mod.append({'Building' : row['Building'],
                            'DayOfWeek' : row['DayOfWeek'], 'StartTime' : row['StartTime'] - interval,
                              'EndTime' : row['EndTime'], 'Name' : build_transitions[row['Building']]["Entry"][0]} , ignore_index=True)
        else:
            for j in build_transitions[row['Building']]["Entry"]:
                entry_mod = entry_mod.append({'Building' : row['Building'],
                            'DayOfWeek' : row['DayOfWeek'], 'StartTime' : row['StartTime'] - interval,
                              'EndTime' : row['EndTime'], 'Name' : j[0]} , ignore_index=True)
for index, row in exit.iterrows():
    name = 'NA'
    if(row["StartTime"] % 100 == 50):
        interval = 50
    else:
        interval = 10
    if(row["Building"] in build_transitions):
        if(len(build_transitions[row['Building']]["Exit"]) == 1):
            exit_mod = exit_mod.append({'Building' : row['Building'],
                            'DayOfWeek' : row['DayOfWeek'], 'StartTime' : row['StartTime'],
                              'EndTime' : row['EndTime'] + interval, 'Name' : build_transitions[row['Building']]["Exit"][0]} , ignore_index=True)
        else:
            for j in build_transitions[row['Building']]["Exit"]:
                exit_mod = exit_mod.append({'Building' : row['Building'],
                            'DayOfWeek' : row['DayOfWeek'], 'StartTime' : row['StartTime'],
                              'EndTime' : (row['EndTime'] + interval), 'Name' : j[0]} , ignore_index=True)
entry_mod.drop_duplicates(keep = 'first', inplace = True)
exit_mod.drop_duplicates(keep = 'first', inplace = True)
entry_mod.to_csv("Entry_mod.csv", index = False)

In [11]:
transition = pd.merge(entry_mod, exit_mod, on = ['Building',"DayOfWeek", "StartTime", "EndTime","Name"], how = "outer")
transition["EventID"] = ["Event_" + id_generator(size = 7) for i in range(transition.shape[0])]
transition["Type"] = "Transition"
transition

,Building,DayOfWeek,StartTime,EndTime,Name,EventID,Type
0,GREENE BUILDING,1.0,650.0,700.0,GREENE FRONT ENT,Event_6202775,Transition
1,GREENE BUILDING,1.0,950.0,1000.0,GREENE FRONT ENT,Event_1042535,Transition
2,GREENE BUILDING,1.0,1150.0,1200.0,GREENE FRONT ENT,Event_0095816,Transition
3,GREENE BUILDING,1.0,1350.0,1400.0,GREENE FRONT ENT,Event_6565454,Transition
4,GREENE BUILDING,1.0,1550.0,1600.0,GREENE FRONT ENT,Event_0109368,Transition
...,...,...,...,...,...,...,...
1452,CARY HALL,4.0,1750.0,1800.0,CARY BACK EXIT,Event_8577489,Transition
1453,CARY HALL,4.0,1550.0,1600.0,CARY BACK EXIT,Event_6125207,Transition
1454,CARY HALL,4.0,2050.0,2100.0,CARY BACK EXIT,Event_8097425,Transition
1455,CARY HALL,5.0,1350.0,1400.0,CARY BACK EXIT,Event_4263274,Transition


In [12]:
test_entry = [800]
x = test_entry[0]
while(x<= 1700):
    x += 30
    test_entry.append(x)
    x+= 70
    test_entry.append(x)
test_exit = test_entry[1:]
test_exit.append(test_entry[-1] + 30)
test_exit
print(len(test_exit)*5)

105


In [13]:
test_events = pd.DataFrame({"EventID": ["Event_" + id_generator(size = 5) for i in range(105)]})
dow = []
start = []
end = []
for day in range(1,6):
    for i in range(len(test_entry)):
        dow.append(day)
        start.append(test_entry[i])
        end.append(test_exit[i])
test_events["DayOfWeek"] = dow
test_events["StartTime"] = start
test_events["EndTime"] = end
test_events["Name"] = "Covid Testing"
test_events["SeatsTaken"] = 0
test_events["Seats"] = 40
test_events["Building"] = "HOUSTON FIELD HOUSE"
test_events["LocationID"] = location.loc[location["Building"] == "HOUSTON FIELD HOUSE", "LocationID"].iloc[0]
test_events["Type"] = "TEST"

test_events

,EventID,DayOfWeek,StartTime,EndTime,Name,SeatsTaken,Seats,Building,LocationID,Type
0,Event_81263,1,800,830,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
1,Event_69122,1,830,900,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
2,Event_23882,1,900,930,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
3,Event_99544,1,930,1000,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
4,Event_46333,1,1000,1030,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
...,...,...,...,...,...,...,...,...,...,...
100,Event_81785,5,1600,1630,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
101,Event_35057,5,1630,1700,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
102,Event_97918,5,1700,1730,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST
103,Event_10627,5,1730,1800,Covid Testing,0,40,HOUSTON FIELD HOUSE,Loc_72581,TEST


In [14]:
for index, row in persons.iterrows():
    select_tests = test_events[test_events["SeatsTaken"] < test_events["Seats"]]
    test_id = np.random.choice(select_tests["EventID"])
    row["Test"] = test_id
    test_events.loc[test_events["EventID"] == test_id, "SeatsTaken"] += 1

In [15]:
persons.head()

,PersonID,Address,Test
0,Person_729960822,Bray Hall,Event_70892
1,Person_890908145,Blitman Residence Commons,Event_10989
2,Person_332261652,Cary Hall,Event_69122
3,Person_076194812,Blitman Residence Commons,Event_69122
4,Person_135788187,Blitman Residence Commons,Event_48018


In [16]:
dorm_events = pd.DataFrame()
name = []
day = []
building = []
start = []
end = []
for i in range(1,6):
    for add in addresses:
        day.append(i)
        day.append(i)
        building.append(add)
        building.append(add)
        start.append(800)
        end.append('NA')
        start.append('NA')
        end.append(2200)
        name.append(add + "_Start")
        name.append(add + "_End")
dorm_events["DayOfWeek"] = day
dorm_events["Name"] = name
dorm_events["EventID"] = (dorm_events.index + 151)
dorm_events["StartTime"] = start
dorm_events["EndTime"] = end
dorm_events["Seats"] = 0
dorm_events["SeatsTaken"] = 1700
dorm_events["Building"] = building
dorm_events["Type"] = "Dorm"
dorm_events

,DayOfWeek,Name,EventID,StartTime,EndTime,Seats,SeatsTaken,Building,Type
0,1,Barton Hall_Start,151,800,NA,0,1700,Barton Hall,Dorm
1,1,Barton Hall_End,152,NA,2200,0,1700,Barton Hall,Dorm
2,1,Bray Hall_Start,153,800,NA,0,1700,Bray Hall,Dorm
3,1,Bray Hall_End,154,NA,2200,0,1700,Bray Hall,Dorm
4,1,Cary Hall_Start,155,800,NA,0,1700,Cary Hall,Dorm
...,...,...,...,...,...,...,...,...,...
65,5,City Station South_End,216,NA,2200,0,1700,City Station South,Dorm
66,5,Polytechnic Apartments_Start,217,800,NA,0,1700,Polytechnic Apartments,Dorm
67,5,Polytechnic Apartments_End,218,NA,2200,0,1700,Polytechnic Apartments,Dorm
68,5,Beman and Brinsmade_Start,219,800,NA,0,1700,Beman and Brinsmade,Dorm


In [17]:
def check_times(sel_times, day_times):
    for day in day_times:
        if(len(day_times[day]) == 0 or len(sel_times[day]) == 0):
            continue
        for tup in day_times[day]:
            startA = sel_times[day][0]
            endA = sel_times[day][1]
            startB = tup[0]
            endB = tup[1]
            if(max(startA, startB) < min(endA, endB) or (startA == startB and endA == endB)):
                #print("NOT VALID")
                #print(str(startA) + " " + str(endA))
                #print(str(startB) + " " + str(endB))
                return False
    return True
print(check_times({1: (), 2: (1030, 1130), 3: (), 4: (1300, 1350), 5: (1000, 1120)},
           {1: {(1000, 1150)}, 2: {(1000, 1150)}, 3: {(900, 950)}, 4: set(), 5: set()}))
            


False


In [18]:

person_event_mon = pd.DataFrame(columns = ["PersonID", "Start", "End", "EventID", "Day"])
person_event_tue = pd.DataFrame(columns = ["PersonID", "Start","End", "EventID", "Day"])
person_event_wed = pd.DataFrame(columns = ["PersonID", "Start","End", "EventID", "Day"])
person_event_thurs = pd.DataFrame(columns = ["PersonID", "Start","End", "EventID", "Day"])
person_event_fri = pd.DataFrame(columns = ["PersonID", "Start","End", "EventID", "Day"])
event_dfs = [person_event_mon, person_event_tue,person_event_wed,
                person_event_thurs, person_event_fri]

In [19]:
other_rows = []
locations = ['Union', 'Library', 'Commons']
for day in range(1,6):
    start = 800
    while start < 1800:
        for loc in locations:
            other_rows.append([day, start, start + 50, loc])
        start += 100
other_events = pd.DataFrame(other_rows, columns = ["DayOfWeek", "StartTime", "EndTime", "Name"])
other_events["EventID"] = (other_events.index +1)
other_events["Seats"] = 0
other_events["SeatsTaken"] = 200
other_events["Type"] = "Other"
other_events

,DayOfWeek,StartTime,EndTime,Name,EventID,Seats,SeatsTaken,Type
0,1,800,850,Union,1,0,200,Other
1,1,800,850,Library,2,0,200,Other
2,1,800,850,Commons,3,0,200,Other
3,1,900,950,Union,4,0,200,Other
4,1,900,950,Library,5,0,200,Other
...,...,...,...,...,...,...,...,...
145,5,1600,1650,Library,146,0,200,Other
146,5,1600,1650,Commons,147,0,200,Other
147,5,1700,1750,Union,148,0,200,Other
148,5,1700,1750,Library,149,0,200,Other


In [20]:
def reg_courses(row):
    crns = []
    day_times = {1: [], 2: [], 3: [],
                 4: [], 5: []}
    student = row["PersonID"]
    test = row["Test"]
    test_day = test_events.loc[test_events["EventID"] == test]["DayOfWeek"].iloc[0]
    test_start = test_events.loc[test_events["EventID"] == test]["StartTime"].iloc[0]
    test_end = test_events.loc[test_events["EventID"] == test]["EndTime"].iloc[0]
    day_times[test_day].append((test_start, test_end))
    test_df = [{'PersonID':student, 'Start': test_start, 'End': test_end, 'EventID': test}]
    event_dfs[test_day-1] = event_dfs[test_day-1].append(test_df, ignore_index = True)
    while(len(crns)<4):
        available_events = events[events["SeatsTaken"] < events["Seats"]]
        crn = np.random.choice(list(available_events["CRN"]))
        sections = available_events[available_events["CRN"] == crn].reset_index(drop = True)
        sel_times = {1: (), 2: (), 3: (), 4: (), 5:()}
        event_ids = []
        for i in range(len(sections)):
            event_ids.append(sections.iloc[i]["EventID"])
            se = (sections.iloc[i]["StartTime"], sections.iloc[i]["EndTime"])
            day = sections.iloc[i]["DayOfWeek"]
            sel_times[day] = se
        if(check_times(sel_times, day_times) == True):  
             crns.append(crn)
             for event in event_ids:
                events.loc[events["EventID"] == event, "SeatsTaken"] += 1
                day = events.loc[events["EventID"] == event]["DayOfWeek"].iloc[0]
                start = events.loc[events["EventID"] == event]["StartTime"].iloc[0]
                end = events.loc[events["EventID"] == event]["EndTime"].iloc[0]
                #event_times[day][start] = event
                temp_df = [{'PersonID':student, 'Start': start, 'End': end, 'EventID': event, 'Day': day}]
                event_dfs[day-1] = event_dfs[day-1].append(temp_df, ignore_index = True)
                if(len(sel_times[day]) > 0):
                    day_times[day].append(sel_times[day])
    for day in day_times:
        filter_day = other_events[other_events["DayOfWeek"] == day]
        if(len(day_times[day]) < 5):
            while (len(day_times[day])<5):
                sel_times = {1: (), 2: (), 3: (), 4: (), 5:()} 
                if(len(filter_day) == 0):
                    break
                other = np.random.choice(filter_day["EventID"])
                other_day = other_events.loc[other_events["EventID"] == other]["DayOfWeek"].iloc[0]
                other_start = other_events.loc[other_events["EventID"] == other]["StartTime"].iloc[0]
                other_end = other_events.loc[other_events["EventID"] == other]["EndTime"].iloc[0]
                sel_times[other_day] = (other_start, other_end)
                if(check_times(sel_times, day_times) == True):
                    temp_df = [{'PersonID':student, 'Start': other_start, 'End': other_end, 'EventID': other}]
                    event_dfs[day-1] = event_dfs[day-1].append(temp_df, ignore_index = True)
                    day_times[day].append(sel_times[day])
                else:
                    filter_day = filter_day[filter_day["StartTime"] != other_start]
        
for index, row in persons.iterrows():
    reg_courses(row)

In [21]:
events = pd.concat([events, other_events], axis = 0)
events = pd.concat((events, test_events), axis = 0)
events.reset_index(drop=True, inplace=True)
events

,CRN,EventID,Section,Name,DayOfWeek,StartTime,EndTime,SeatsTaken,Seats,LocationID,Building,Type
0,81215.0,Event_1943344516,01,Air And Space Studies 300 A,1,700,950,2,13,Loc_81644,GREENE BUILDING,Class
1,85313.0,Event_7152077997,01,Programming For Bme,1,1000,1150,7,13,Loc_81644,GREENE BUILDING,Class
2,81136.0,Event_4786369640,01,Introduction To Optimization,1,1000,1150,2,13,Loc_81644,GREENE BUILDING,Class
3,85230.0,Event_3240167821,01,Sedimentology /Stratigraphy,1,1200,1350,5,13,Loc_81644,GREENE BUILDING,Class
4,84417.0,Event_6000328461,03,Architectural Design Studio 7,1,1400,1750,2,13,Loc_81644,GREENE BUILDING,Class
...,...,...,...,...,...,...,...,...,...,...,...,...
3306,NaN,Event_81785,NaN,Covid Testing,5,1600,1630,11,40,Loc_72581,HOUSTON FIELD HOUSE,TEST
3307,NaN,Event_35057,NaN,Covid Testing,5,1630,1700,16,40,Loc_72581,HOUSTON FIELD HOUSE,TEST
3308,NaN,Event_97918,NaN,Covid Testing,5,1700,1730,19,40,Loc_72581,HOUSTON FIELD HOUSE,TEST
3309,NaN,Event_10627,NaN,Covid Testing,5,1730,1800,13,40,Loc_72581,HOUSTON FIELD HOUSE,TEST


In [22]:
# Define schedule columns

# Week starts at unkwown location defined by ID 0
week_start = []
monday_1_entry = []
monday_1 = []
monday_1_exit = []
monday_2_entry = []
monday_2 = []
monday_2_exit = []
monday_3_entry = []
monday_3 = []
monday_3_exit = []
monday_4_entry = []
monday_4 = []
monday_4_exit = []
monday_5_entry = []
monday_5 = []
monday_5_exit = []

# Monday ends at unknown location defined by ID 6
monday_end = []
tuesday_1_entry = []
tuesday_1 = []
tuesday_1_exit = []
tuesday_2_entry = []
tuesday_2 = []
tuesday_2_exit = []
tuesday_3_entry = []
tuesday_3 = []
tuesday_3_exit = []
tuesday_4_entry = []
tuesday_4 = []
tuesday_4_exit = []
tuesday_5_entry = []
tuesday_5 = []
tuesday_5_exit = []

# Tuesday ends at unknown location defined by ID 10
tuesday_end = []
wednesday_1_entry = []
wednesday_1 = []
wednesday_1_exit = []
wednesday_2_entry = []
wednesday_2 = []
wednesday_2_exit = []
wednesday_3_entry = []
wednesday_3 = []
wednesday_3_exit = []
wednesday_4_entry = []
wednesday_4 = []
wednesday_4_exit = []
wednesday_5_entry = []
wednesday_5 = []
wednesday_5_exit = []

# Wednesday ends at unknown location defined by ID 15
wednesday_end = []
thursday_1_entry = []
thursday_1 = []
thursday_1_exit = []
thursday_2_entry = []
thursday_2 = []
thursday_2_exit = []
thursday_3_entry = []
thursday_3 = []
thursday_3_exit = []
thursday_4_entry = []
thursday_4 = []
thursday_4_exit = []
thursday_5_entry = []
thursday_5 = []
thursday_5_exit = []

# Thursday ends at unknown location defined by ID 18
thursday_end = []
friday_1_entry = []
friday_1 = []
friday_1_exit = []
friday_2_entry = []
friday_2 = []
friday_2_exit = []
friday_3_entry = []
friday_3 = []
friday_3_exit = []
friday_4_entry = []
friday_4 = []
friday_4_exit = []
friday_5_entry = []
friday_5 = []
friday_5_exit = []

# Week ends at unkwown location defined by ID 22
week_end = []

In [23]:
def schedule_day(day_num, slots):
    for index, row in persons.iterrows():
        day = event_dfs[day_num]
        stu_events_day = (day[day["PersonID"] == row["PersonID"]])
        stu_events_day = stu_events_day.sort_values(by = ['Start'])
        stu_events_day = stu_events_day.reset_index(drop = True)
        for i in range(5):
            if((i > len(stu_events_day)-1)):
                event = slots[i]
                event[0].append('NA')
                event[1].append('NA')
                event[2].append('NA')
                continue
            event = slots[i]
            event_id = stu_events_day.iloc[i]["EventID"]
            event_start = stu_events_day.iloc[i]["Start"]
            event_end = stu_events_day.iloc[i]["End"]
            if(event_end % 100 == 50):
                tran_time = 50
            else:
                tran_time = 10
            if(event_start % 100 == 0):
                tran_time2 = 50
            else:
                tran_time2 = 10
            location_id = events.loc[events["EventID"] == event_id, "LocationID"].iloc[0]
            event[1].append(event_id)
            if((location_id != "NA") and (pd.isnull(location_id) == False)):
                building = location[location["LocationID"] == location_id]["Building"].iloc[0]
                if(building in build_transitions):
                    if(len(build_transitions[building]["Entry"]) > 1):
                        df_entry = pd.DataFrame(build_transitions[building]["Entry"], columns = ["Name", "Prob"])
                        df_exit = pd.DataFrame(build_transitions[building]["Exit"], columns = ["Name", "Prob"])
                        entry_choice = np.random.choice(df_entry["Name"], 1, p = df_entry["Prob"])
                        exit_choice = np.random.choice(df_exit["Name"], 1, p = df_exit["Prob"])
                        sel_event1 = transition.loc[(transition["Name"] == entry_choice[0]) & (transition["StartTime"] == event_start - tran_time2) & (transition["EndTime"] == event_start)]["EventID"].iloc[0]
                        sel_event2 = transition.loc[(transition["Name"] == exit_choice[0]) & (transition["StartTime"] == event_end) & (transition["EndTime"] == (event_end+tran_time))]["EventID"].iloc[0]
                    else:
                        sel_event1 = transition.loc[(transition["Building"] == building) & (transition["StartTime"] == event_start - tran_time2) & (transition["EndTime"] == event_start)]["EventID"].iloc[0]
                        sel_event2 = transition.loc[(transition["Building"] == building) & (transition["StartTime"] == event_end) & (transition["EndTime"] == (event_end+tran_time))]["EventID"].iloc[0]
                    event[0].append(sel_event1)
                    event[2].append(sel_event2)
                else:
                    event[0].append('NA')
                    event[2].append('NA')
            else:
                event[0].append('NA')
                event[2].append('NA')  

    

In [24]:
def add_dorm_events(day, slot, time):
    day_events = dorm_events[dorm_events["DayOfWeek"] == day]
    for index, row in persons.iterrows():
        event = day_events[day_events["Building"] == row["Address"]]
        if(time):
            event = event[event["StartTime"] != 'NA']
        else:
            event = event[event["EndTime"] != 'NA']
        slot.append(event["EventID"].iloc[0])

In [25]:
add_dorm_events(1, week_start, True)
add_dorm_events(1, monday_end, False)
add_dorm_events(2, tuesday_end, False)
add_dorm_events(3, wednesday_end, False)
add_dorm_events(4, thursday_end, False)
add_dorm_events(5, week_end, False)

In [30]:
schedule_day(0, [[monday_1_entry, monday_1, monday_1_exit], [monday_2_entry,
          monday_2, monday_2_exit], [monday_3_entry, monday_3, monday_3_exit],
          [monday_4_entry, monday_4, monday_4_exit], [monday_5_entry, monday_5, monday_5_exit]])

In [31]:
schedule_day(1, [[tuesday_1_entry, tuesday_1, tuesday_1_exit], [tuesday_2_entry,
          tuesday_2, tuesday_2_exit], [tuesday_3_entry, tuesday_3, tuesday_3_exit],
          [tuesday_4_entry, tuesday_4, tuesday_4_exit], [tuesday_5_entry, tuesday_5, tuesday_5_exit]])

In [32]:
schedule_day(2, [[wednesday_1_entry, wednesday_1, wednesday_1_exit], [wednesday_2_entry,
          wednesday_2, wednesday_2_exit], [wednesday_3_entry, wednesday_3, wednesday_3_exit],
          [wednesday_4_entry, wednesday_4, wednesday_4_exit], [wednesday_5_entry, wednesday_5, wednesday_5_exit]])

In [33]:
schedule_day(3, [[thursday_1_entry, thursday_1, thursday_1_exit], [thursday_2_entry,
          thursday_2, thursday_2_exit], [thursday_3_entry, thursday_3, thursday_3_exit],
          [thursday_4_entry, thursday_4, thursday_4_exit], [thursday_5_entry, thursday_5, thursday_5_exit]])

In [34]:
schedule_day(4, [[friday_1_entry, friday_1, friday_1_exit], [friday_2_entry,
          friday_2, friday_2_exit], [friday_3_entry, friday_3, friday_3_exit],
          [friday_4_entry, friday_4, friday_4_exit], [friday_5_entry, friday_5, friday_5_exit]])

In [35]:
len(friday_5)

1700

In [36]:
schedule = pd.DataFrame({"student_id": persons["PersonID"], "week_start": week_start, "monday_1_entry": monday_1_entry,
                         "monday_1": monday_1, "monday_1_exit": monday_1_exit, "monday_2_entry": monday_2_entry, "monday_2": monday_2,
                         "monday_2_exit": monday_2_exit, "monday_3_entry": monday_3_entry, "monday_3": monday_3, "monday_3_exit": monday_3_exit,
                        "monday_4_entry": monday_4_entry, "monday_4": monday_4, "monday_4_exit": monday_4_exit,
                         "monday_5_entry": monday_5_entry, "monday_5": monday_5, "monday_5_exit": monday_5_exit,"monday_end": monday_end,
                        "tuesday_1_entry": tuesday_1_entry,
                         "tuesday_1": tuesday_1, "tuesday_1_exit": tuesday_1_exit, "tuesday_2_entry": tuesday_2_entry, "tuesday_2": tuesday_2,
                         "tuesday_2_exit": tuesday_2_exit, "tuesday_3_entry": tuesday_3_entry, "tuesday_3": tuesday_3, "tuesday_3_exit": tuesday_3_exit,
                        "tuesday_4_entry": tuesday_4_entry, "tuesday_4": tuesday_4, "tuesday_4_exit": tuesday_4_exit,
                         "tuesday_5_entry": tuesday_5_entry, "tuesday_5": tuesday_5, "tuesday_5_exit": tuesday_5_exit,"tuesday_end": tuesday_end,
                        "wednesday_1_entry": wednesday_1_entry,
                         "wednesday_1": wednesday_1, "wednesday_1_exit": wednesday_1_exit, "wednesday_2_entry": wednesday_2_entry, "wednesday_2": wednesday_2,
                         "wednesday_2_exit": wednesday_2_exit, "wednesday_3_entry": wednesday_3_entry, "wednesday_3": wednesday_3, "wednesday_3_exit": wednesday_3_exit,
                        "wednesday_4_entry": wednesday_4_entry, "wednesday_4": wednesday_4, "wednesday_4_exit": wednesday_4_exit,
                         "wednesday_5_entry": wednesday_5_entry, "wednesday_5": wednesday_5, "wednesday_5_exit": wednesday_5_exit,"wednesday_end": wednesday_end,
                        "thursday_1_entry": thursday_1_entry,
                         "thursday_1": thursday_1, "thursday_1_exit": thursday_1_exit, "thursday_2_entry": thursday_2_entry, "thursday_2": thursday_2,
                         "thursday_2_exit": thursday_2_exit, "thursday_3_entry": thursday_3_entry, "thursday_3": thursday_3, "thursday_3_exit": thursday_3_exit,
                        "thursday_4_entry": thursday_4_entry, "thursday_4": thursday_4, "thursday_4_exit": thursday_4_exit,
                         "thursday_5_entry": thursday_5_entry, "thursday_5": thursday_5, "thursday_5_exit": thursday_5_exit,"thursday_end": thursday_end,
                        "friday_1_entry": friday_1_entry,
                         "friday_1": friday_1, "friday_1_exit": friday_1_exit, "friday_2_entry": friday_2_entry, "friday_2": friday_2,
                         "friday_2_exit": friday_2_exit, "friday_3_entry": friday_3_entry, "friday_3": friday_3, "friday_3_exit": friday_3_exit,
                        "friday_4_entry": friday_4_entry, "friday_4": friday_4, "friday_4_exit": friday_4_exit,
                         "friday_5_entry": friday_5_entry, "friday_5": friday_5, "friday_5_exit": friday_5_exit,"week_end": week_end
                        })
schedule

,student_id,week_start,monday_1_entry,monday_1,monday_1_exit,monday_2_entry,monday_2,monday_2_exit,monday_3_entry,monday_3,...,friday_3_entry,friday_3,friday_3_exit,friday_4_entry,friday_4,friday_4_exit,friday_5_entry,friday_5,friday_5_exit,week_end
0,Person_729960822,153,NA,3,NA,NA,4,NA,NA,10,...,NA,138,NA,Event_7294418,Event_0612317893,Event_3556068,NA,149,NA,210
1,Person_890908145,157,NA,2,NA,NA,9,NA,Event_0249277,Event_7543626146,...,NA,137,NA,NA,140,NA,NA,144,NA,214
2,Person_332261652,155,NA,Event_69122,NA,NA,6,NA,Event_4553144,Event_2526613808,...,NA,130,NA,Event_1878648,Event_9517533515,Event_4423353,NA,139,NA,212
3,Person_076194812,157,NA,Event_69122,NA,NA,10,NA,Event_4202693,Event_6157185134,...,Event_1795146,Event_6253430186,Event_0255533,NA,139,NA,NA,150,NA,214
4,Person_135788187,157,NA,6,NA,Event_5671531,Event_7527081792,Event_6842677,Event_8639659,Event_2219351269,...,NA,144,NA,NA,146,NA,NA,150,NA,214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,Person_530153644,161,NA,Event_69122,NA,NA,11,NA,NA,14,...,Event_9694406,Event_2459012407,Event_3556068,NA,145,NA,NA,148,NA,218
1696,Person_731312028,153,NA,3,NA,NA,11,NA,Event_7679381,Event_4426332462,...,NA,135,NA,NA,140,NA,NA,147,NA,210
1697,Person_220410115,155,Event_3689147,Event_4319106639,Event_4169224,NA,8,NA,Event_8639659,Event_7538105696,...,Event_6663553,Event_8263807962,Event_8683731,NA,143,NA,NA,150,NA,212
1698,Person_079255674,163,Event_0098348,Event_6999662533,Event_2204182,NA,15,NA,NA,18,...,Event_7033897,Event_4654956987,Event_8639659,Event_3806475,Event_4440946876,Event_1921454,NA,150,NA,220


In [37]:
schedule.to_csv("SCHEDULE.csv", index = False)

In [38]:
personID = []
eventID = []

for index, row in schedule.iterrows():
    for i in range(81):
        personID.append(row[0])
        eventID.append(row[i+1])

attends = pd.DataFrame({"PersonID": personID, "EventID": eventID})
attends = attends.merge(events, how = 'inner', left_on= 'EventID', 
                        right_on='EventID').drop(["Name", "DayOfWeek", "SeatsTaken", "Seats", "LocationID"], axis = 1)
attends["Role"] = "Student"
attends["RemoteOrLive"] = "Live"

attends = attends[["PersonID", "EventID", "Role", "RemoteOrLive", "StartTime", "EndTime"]]
attends.columns = ["PersonID", "EventID", "Role", "RemoteOrLive", "ArrivalTime", "DepartureTime"]

attends.head()

,PersonID,EventID,Role,RemoteOrLive,ArrivalTime,DepartureTime
0,Person_729960822,3,Student,Live,800,850
1,Person_397524382,3,Student,Live,800,850
2,Person_806900142,3,Student,Live,800,850
3,Person_379890516,3,Student,Live,800,850
4,Person_861843422,3,Student,Live,800,850


In [39]:
attends.to_csv("ATTENDS.csv", index = False)

In [40]:
# Get all contacts
contacts = attends.merge(attends, how = 'inner', on = 'EventID')[
    ["PersonID_x", "PersonID_y", "ArrivalTime_x", "DepartureTime_x", "EventID"]
]

#contacts['Type'] = "SameRoom"
# Merge with events to get time information

contacts = contacts.merge(events, 
               how = 'left', 
               on = 'EventID')[["PersonID_x", "PersonID_y", "DayOfWeek", 
                                 "ArrivalTime_x", "DepartureTime_x", "Type", "Building"]]


# Rename columns
contacts.columns = ["PersonID1", "PersonID2", "Date", "StartTime", "EndTime", "Type", "Building"]

#contacts[''] = np.where(contacts['Building'].isnull() == True, 'SameRoom', 'SameTransition')
contacts_mod = contacts[contacts["Type"] == "Class"]
contacts_mod.head()

,PersonID1,PersonID2,Date,StartTime,EndTime,Type,Building
108282,Person_729960822,Person_729960822,1,1400,1750,Class,JONSSON ENGINEERING CENTER
108283,Person_729960822,Person_962337174,1,1400,1750,Class,JONSSON ENGINEERING CENTER
108284,Person_729960822,Person_468673850,1,1400,1750,Class,JONSSON ENGINEERING CENTER
108285,Person_729960822,Person_117470952,1,1400,1750,Class,JONSSON ENGINEERING CENTER
108286,Person_729960822,Person_043255129,1,1400,1750,Class,JONSSON ENGINEERING CENTER


In [42]:
contacts_class = contacts[contacts["Type"] == "Class"]
contacts_test = contacts[contacts["Type"] == "TEST"]
contacts_other = contacts[contacts["Type"] == "Other"]
contacts_class.to_csv("CONTACTS_CLASS.csv", index = False)
contacts_test.to_csv("CONTACTS_TEST.csv", index = False)
contacts_other.to_csv("CONTACTS_OTHER.csv", index = False)

In [47]:
precedes = pd.DataFrame({"EventID1" : week_start + monday_1 + monday_2 + monday_3 + 
    monday_4 + monday_5 +
    monday_end + tuesday_1 + tuesday_2 + 
    tuesday_3 + tuesday_4 + tuesday_5 + tuesday_end + wednesday_1 + wednesday_2 +
    wednesday_3 + wednesday_4 + wednesday_5 + wednesday_end + thursday_1 +
    thursday_2 + thursday_3 + thursday_4 + thursday_5 + thursday_end + friday_1 + friday_2 +
    friday_3 + friday_4 + friday_5,
                         "EventID2": monday_1 + monday_2 + monday_3 + 
    monday_4 + monday_5 +
    monday_end + tuesday_1 + tuesday_2 + 
    tuesday_3 + tuesday_4 + tuesday_5 + tuesday_end + wednesday_1 + wednesday_2 +
    wednesday_3 + wednesday_4 + wednesday_5 + wednesday_end + thursday_1 +
    thursday_2 + thursday_3 + thursday_4 + thursday_5 + thursday_end + friday_1 + friday_2 +
    friday_3 + friday_4 + friday_5 + week_end})
precedes = precedes.groupby(['EventID1', 'EventID2']).size().reset_index()
precedes.columns = ["EventID1", "EventID2", "HeadCount"]
precedes = precedes[precedes.EventID2 != "NA"]
precedes = precedes[precedes.EventID1 != "NA"]
precedes

,EventID1,EventID2,HeadCount
0,1,4,20
1,1,5,26
2,1,6,23
3,1,7,9
4,1,8,12
...,...,...,...
26781,Event_9999433297,56,1
26782,Event_9999433297,59,1
26783,Event_9999433297,168,1
26784,Event_9999433297,172,1


In [44]:
precedes.to_csv("PRECEDES.csv", index = False)

In [45]:
events = pd.concat((events, transition), axis = 0)
events = pd.concat((events, dorm_events), axis = 0)
events.reset_index(drop=True, inplace=True)
events

,CRN,EventID,Section,Name,DayOfWeek,StartTime,EndTime,SeatsTaken,Seats,LocationID,Building,Type
0,81215.0,Event_1943344516,01,Air And Space Studies 300 A,1.0,700,950,2.0,13.0,Loc_81644,GREENE BUILDING,Class
1,85313.0,Event_7152077997,01,Programming For Bme,1.0,1000,1150,7.0,13.0,Loc_81644,GREENE BUILDING,Class
2,81136.0,Event_4786369640,01,Introduction To Optimization,1.0,1000,1150,2.0,13.0,Loc_81644,GREENE BUILDING,Class
3,85230.0,Event_3240167821,01,Sedimentology /Stratigraphy,1.0,1200,1350,5.0,13.0,Loc_81644,GREENE BUILDING,Class
4,84417.0,Event_6000328461,03,Architectural Design Studio 7,1.0,1400,1750,2.0,13.0,Loc_81644,GREENE BUILDING,Class
...,...,...,...,...,...,...,...,...,...,...,...,...
4833,NaN,216,NaN,City Station South_End,5.0,NA,2200,1700.0,0.0,NaN,City Station South,Dorm
4834,NaN,217,NaN,Polytechnic Apartments_Start,5.0,800,NA,1700.0,0.0,NaN,Polytechnic Apartments,Dorm
4835,NaN,218,NaN,Polytechnic Apartments_End,5.0,NA,2200,1700.0,0.0,NaN,Polytechnic Apartments,Dorm
4836,NaN,219,NaN,Beman and Brinsmade_Start,5.0,800,NA,1700.0,0.0,NaN,Beman and Brinsmade,Dorm


In [46]:
events.to_csv("EVENT.csv", index = False)